# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-11-2022'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-11-2022'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-02-12 04:21:06,33.93911,67.709953,169940,7478,NaN,NaN,Afghanistan,436.545703,4.400377
1,NaN,NaN,NaN,Albania,2022-02-12 04:21:06,41.15330,20.168300,267020,3402,NaN,NaN,Albania,9278.615609,1.274062
2,NaN,NaN,NaN,Algeria,2022-02-12 04:21:06,28.03390,1.659600,260723,6703,NaN,NaN,Algeria,594.565105,2.570928
3,NaN,NaN,NaN,Andorra,2022-02-12 04:21:06,42.50630,1.521800,37140,148,NaN,NaN,Andorra,48068.336245,0.398492
4,NaN,NaN,NaN,Angola,2022-02-12 04:21:06,-11.20270,17.873900,98501,1898,NaN,NaN,Angola,299.702418,1.926884


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,164190,164727,165358,165711,166191,166924,167739,168550,169448,169940
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,261240,261240,263172,263172,264624,264875,265716,266416,267020,267020
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,254885,255836,256806,257598,257976,258478,259088,259673,260191,260723


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7418,7420,7420,7425,7432,7442,7454,7465,7477,7478
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3362,3362,3371,3371,3380,3387,3393,3399,3402,3402
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6604,6618,6631,6640,6646,6654,6667,6678,6690,6703


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,14826,14826,14970,14999,14999,15046,15089,15121,15155,15216
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,53083,53083,53559,53637,53637,53768,53992,54095,54203,54301
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5297,5297,5338,5346,5346,5356,5372,5380,5391,5402


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,168,168,170,170,170,171,171,173,173,173
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,616,616,623,623,623,624,624,626,626,626
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,85,85,85,85,85,85,85,85,87,87


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
674,1001.0,Autauga,Alabama,US,2022-02-12 04:21:06,32.539527,-86.644082,15216,173,NaN,NaN,"Autauga, Alabama, US",27235.139344,1.136961
711,1075.0,Lamar,Alabama,US,2022-02-12 04:21:06,33.779950,-88.096680,3527,62,NaN,NaN,"Lamar, Alabama, US",25548.714234,1.757868
712,1077.0,Lauderdale,Alabama,US,2022-02-12 04:21:06,34.901719,-87.656247,23502,348,NaN,NaN,"Lauderdale, Alabama, US",25344.822008,1.480725


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,77652197,918451,0.0
India,42586544,507981,0.0
Brazil,27299336,637467,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,77652197,918451,0.0,76733746.0,2022-02-12 04:21:06,37.934269,-91.444083
India,42586544,507981,0.0,42078563.0,2022-02-12 04:21:06,23.088275,81.806127
Brazil,27299336,637467,0.0,26661869.0,2022-02-12 04:21:06,-12.669522,-48.480493
France,21646561,135534,0.0,21511027.0,2022-02-12 04:21:06,6.430808,-34.730285
United Kingdom,18346553,159909,0.0,18186644.0,2022-02-12 04:21:06,28.164647,-33.966498


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,8799757,82418,0.0
Texas,6498479,81897,0.0
Florida,5776333,67572,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,8799757,82418,0.0,8717339.0,2022-02-12 04:21:06,37.843962,-120.728594
Texas,6498479,81897,0.0,6416582.0,2022-02-12 04:21:06,31.660643,-98.653069
Florida,5776333,67572,0.0,5708761.0,2022-02-12 04:21:06,28.940755,-82.700744
New York,4878304,66397,0.0,4811907.0,2022-02-12 04:21:06,42.544151,-75.474183
Illinois,2987502,34228,0.0,2953274.0,2022-02-12 04:21:06,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,2752398,29764,0.0
Arizona,Maricopa,1225561,15256,0.0
Florida,Miami-Dade,1168806,9993,0.0
Illinois,Cook,1097792,13725,0.0
Texas,Harris,977268,10410,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,2752398,29764,0.0,2722634.0,2022-02-12 04:21:06,34.308284,-118.228241,6037.0
Arizona,Maricopa,1225561,15256,0.0,1210305.0,2022-02-12 04:21:06,33.348359,-112.491815,4013.0
Florida,Miami-Dade,1168806,9993,0.0,1158813.0,2022-02-12 04:21:06,25.611236,-80.551706,12086.0
Illinois,Cook,1097792,13725,0.0,1084067.0,2022-02-12 04:21:06,41.841448,-87.816588,17031.0
Texas,Harris,977268,10410,0.0,966858.0,2022-02-12 04:21:06,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,168550,266416,259673,36989,98453,11,6853,8675327,396885,2822873,...,749554,231179,7,499537,2404651,594037,465,11579,308050,230740
2022-02-10,169448,267020,260191,37074,98474,11,7321,8700437,399727,2854123,...,758366,231871,7,501586,2430683,600128,465,11604,308556,231040
2022-02-11,169940,267020,260723,37140,98501,11,7331,8716940,402403,2876687,...,765960,232496,7,501586,2457170,600128,490,11659,309013,231214


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,7465,3399,6678,148,1896,0,131,123444,8108,4422,...,6696,1592,1,5504,38614,5196,0,2090,3934,5367
2022-02-10,7477,3402,6690,148,1897,0,134,123707,8123,4479,...,6710,1595,1,5521,38688,5215,0,2093,3935,5373
2022-02-11,7478,3402,6703,148,1898,0,134,123859,8145,4543,...,6729,1597,1,5521,38784,5215,0,2100,3936,5374


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,14826,14826,14970,14999,14999,15046,15089,15121,15155,15216
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,3430,3430,3464,3465,3465,3475,3489,3514,3520,3527
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,22783,22783,23060,23098,23098,23167,23249,23366,23420,23502
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,7158,7158,7248,7261,7261,7285,7302,7327,7354,7384
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,36514,36514,36758,36784,36784,36871,36955,37018,37059,37125


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-09,1254032,230235,18,1931642,800373,8778067,1282919,710669,252068,49,...,1947865,6466862,907174,108166,15128,1593675,1391026,468391,1548219,151803
2022-02-10,1256745,230235,18,1931642,802198,8775650,1285987,711695,252567,49,...,1954202,6484110,909109,108577,15150,1598402,1391026,470807,1551117,152206
2022-02-11,1259580,232102,18,1941802,804145,8799757,1288600,712884,253010,49,...,1960266,6498479,910900,108969,15171,1602677,1410596,472928,1553551,152581


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-02-09       15121   54095    5380  6267  14447    2261   4962   30989   
2022-02-10       15155   54203    5391  6283  14493    2264   4974   31059   
2022-02-11       15216   54301    5402  6294  14516    2267   4985   31148   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-02-09         8332     4969  ...    6588   1862     7886     1890   
2022-02-10         8342     4981  ...    6603   1863     7893     1894   
2022-02-11         8349     4989  ...    6616   1865     7917     1898   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-09          10752  9539  5553          0     2260   1521  
2022-02-10          10790  9560  5564          0     2268   1523  
2022-02-11          10817  9578  5569          0     2274   1524  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-09,17452,1114,0,26856,9927,81884,11538,10214,2598,0,...,23149,81359,4237,562,104,17305,11194,5939,12786,1667
2022-02-10,17502,1114,0,26856,9937,82122,11583,10219,2601,0,...,23212,81655,4250,565,105,17393,11194,5966,12843,1667
2022-02-11,17505,1114,0,27073,9977,82418,11601,10260,2601,0,...,23261,81897,4261,566,105,17482,11316,5992,12877,1667


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-02-09         173     626      85   96    212      49    110     565   
2022-02-10         173     626      87   97    212      49    111     566   
2022-02-11         173     626      87   97    212      49    111     566   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-02-09          149       70  ...     130     39       68       26   
2022-02-10          149       72  ...     130     39       68       26   
2022-02-11          149       72  ...     130     39       68       26   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-09            119    15    36          0       42     17  
2022-02-10            119    15    36          0       42     17  
2022-02-11            119    15    36          0       42     17  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,0.004835,0.002634,0.002258,0.004917,0.000295,0.0,0.000000,0.003151,0.007133,0.004084,...,0.010924,0.003377,0.0,0.003133,0.010063,0.013101,0.000000,0.002250,0.001346,0.000594
2022-02-10,0.005328,0.002267,0.001995,0.002298,0.000213,0.0,0.068291,0.002894,0.007161,0.011070,...,0.011756,0.002993,0.0,0.004102,0.010826,0.010254,0.000000,0.002159,0.001643,0.001300
2022-02-11,0.002904,0.000000,0.002045,0.001780,0.000274,0.0,0.001366,0.001897,0.006695,0.007906,...,0.010014,0.002695,0.0,0.000000,0.010897,0.000000,0.053763,0.004740,0.001481,0.000753


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,0.001476,0.001768,0.001650,0.006803,0.000000,NaN,0.000000,0.001761,0.000741,0.011205,...,0.002996,0.002519,0.0,0.001638,0.002414,0.002895,NaN,0.000958,0.000254,0.000186
2022-02-10,0.001608,0.000883,0.001797,0.000000,0.000527,NaN,0.022901,0.002131,0.001850,0.012890,...,0.002091,0.001884,0.0,0.003089,0.001916,0.003657,NaN,0.001435,0.000254,0.001118
2022-02-11,0.000134,0.000000,0.001943,0.000000,0.000527,NaN,0.000000,0.001229,0.002708,0.014289,...,0.002832,0.001254,0.0,0.000000,0.002481,0.000000,NaN,0.003344,0.000254,0.000186


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-09,0.002918,0.009825,0.0,0.002804,0.002928,0.001407,0.002392,0.001477,0.001585,0.0,...,0.003357,0.002389,0.002278,0.004523,0.001987,0.002982,0.005962,0.004861,0.002045,0.004260
2022-02-10,0.002163,0.000000,0.0,0.000000,0.002280,-0.000275,0.002391,0.001444,0.001980,0.0,...,0.003253,0.002667,0.002133,0.003800,0.001454,0.002966,0.000000,0.005158,0.001872,0.002655
2022-02-11,0.002256,0.008109,0.0,0.005260,0.002427,0.002747,0.002032,0.001671,0.001754,0.0,...,0.003103,0.002216,0.001970,0.003610,0.001386,0.002675,0.014069,0.004505,0.001569,0.002464


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-09      0.002121  0.001908  0.001489  0.003684  0.002081  0.000442   
2022-02-10      0.002249  0.001996  0.002045  0.002553  0.003184  0.001327   
2022-02-11      0.004025  0.001808  0.002040  0.001751  0.001587  0.001325   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-09      0.007513  0.004278  0.001202  0.003636  ...  0.002435   
2022-02-10      0.002418  0.002259  0.001200  0.002415  ...  0.002277   
2022-02-11      0.002211  0.002866  0.000839  0.001606  ...  0.001969   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-09      0.004315  0.002288  0.005854   0.006270  0.003894  0.002528   
2022-02-10      0.000537  0.000888  0.002116   0.003534  0.002201  0.001981   
2022-02-11      0.001074  0.003041  0.002112   0.002502  0.001883  0.000899   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-02-09            NaN  0.009379  0.004624  
2022-02-10            NaN  0.003540  0.001315  
2022-02-11            NaN  0.002646  0.000657  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-09,0.002585,0.019213,NaN,0.001268,0.003234,0.004206,0.010687,0.005513,0.001928,NaN,...,0.004949,0.003639,0.003315,0.007168,0.000000,0.004528,0.008559,0.006610,0.003689,0.0
2022-02-10,0.002865,0.000000,NaN,0.000000,0.001007,0.002907,0.003900,0.000490,0.001155,NaN,...,0.002721,0.003638,0.003068,0.005338,0.009615,0.005085,0.000000,0.004546,0.004458,0.0
2022-02-11,0.000171,0.000000,NaN,0.008080,0.004025,0.003604,0.001554,0.004012,0.000000,NaN,...,0.002111,0.002964,0.002588,0.001770,0.000000,0.005117,0.010899,0.004358,0.002647,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2022-02-09      0.011696  0.003205  0.000000  0.000000  0.004739     0.0   
2022-02-10      0.000000  0.000000  0.023529  0.010417  0.000000     0.0   
2022-02-11      0.000000  0.000000  0.000000  0.000000  0.000000     0.0   

Province_State                                        ... Wyoming         \
Admin2            Butler  Calhoun Chambers  Cherokee  ...    Park Platte   
2022-02-09      0.000000  0.00000      0.0  0.000000  ...     0.0    0.0   
2022-02-10      0.009091  0.00177      0.0  0.028571  ...     0.0    0.0   
2022-02-11      0.000000  0.00000      0.0  0.000000  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2022-02-09          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2022-02-10          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2022-02-11          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2022-02-09        0.0  
2022-02-10        0.0  
2022-02-11        0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,0.004553,0.002746,0.002282,0.003314,0.000267,0.0,0.001199,0.003224,0.006736,0.007292,...,0.010933,0.003490,5.782412e-19,0.002868,0.008942,0.015200,0.018672,0.008777,0.001107,0.000558
2022-02-10,0.004941,0.002507,0.002139,0.002806,0.000240,0.0,0.034745,0.003059,0.006948,0.009181,...,0.011345,0.003241,2.891206e-19,0.003485,0.009884,0.012727,0.009336,0.005468,0.001375,0.000929
2022-02-11,0.003922,0.001253,0.002092,0.002293,0.000257,0.0,0.018056,0.002478,0.006821,0.008544,...,0.010679,0.002968,1.445603e-19,0.001742,0.010390,0.006363,0.031550,0.005104,0.001428,0.000841


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,0.001394,0.001818,0.001624,0.003482,0.000005,NaN,0.001970,0.001885,0.000785,0.013014,...,0.002954,0.001967,0.0,0.001335,0.002460,0.003134,NaN,0.008035,0.000278,0.000323
2022-02-10,0.001501,0.001350,0.001711,0.001741,0.000266,NaN,0.012435,0.002008,0.001317,0.012952,...,0.002522,0.001926,0.0,0.002212,0.002188,0.003395,NaN,0.004735,0.000266,0.000720
2022-02-11,0.000817,0.000675,0.001827,0.000870,0.000397,NaN,0.006218,0.001618,0.002013,0.013620,...,0.002677,0.001590,0.0,0.001106,0.002335,0.001698,NaN,0.004040,0.000260,0.000453


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-09,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-02-10,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-02-11,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-09,0.002941,0.006997,3.814697e-07,0.002673,0.002772,0.003759,0.002543,0.001665,0.001372,2.070976e-210,...,0.004221,0.002553,0.002391,0.003816,0.002182,0.003343,0.004581,0.004241,0.002092,0.003759
2022-02-10,0.002552,0.003498,1.907349e-07,0.001337,0.002526,0.001742,0.002467,0.001555,0.001676,1.035488e-210,...,0.003737,0.002610,0.002262,0.003808,0.001818,0.003154,0.002290,0.004700,0.001982,0.003207
2022-02-11,0.002404,0.005804,9.536743e-08,0.003298,0.002477,0.002244,0.002250,0.001613,0.001715,5.177441e-211,...,0.003420,0.002413,0.002116,0.003709,0.001602,0.002915,0.008180,0.004602,0.001776,0.002835


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-09      0.002434  0.002521  0.001924  0.003142  0.002507  0.000708   
2022-02-10      0.002341  0.002259  0.001984  0.002847  0.002846  0.001017   
2022-02-11      0.003183  0.002033  0.002012  0.002299  0.002216  0.001171   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-09      0.005071  0.003874  0.001519  0.003372  ...  0.002493   
2022-02-10      0.003745  0.003066  0.001360  0.002894  ...  0.002385   
2022-02-11      0.002978  0.002966  0.001099  0.002250  ...  0.002177   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-09      0.004190  0.002768  0.004189   0.005044  0.003179  0.002763   
2022-02-10      0.002364  0.001828  0.003153   0.004289  0.002690  0.002372   
2022-02-11      0.001719  0.002434  0.002632   0.003396  0.002287  0.001635   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-02-09           -1.0  0.005447  0.004467  
2022-02-10           -1.0  0.004493  0.002891  
2022-02-11           -1.0  0.003569  0.001774  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-09,0.001862,0.009636,NaN,0.002587,0.003226,0.003298,0.007902,0.002869,0.002856,NaN,...,0.003190,0.003029,0.004197,0.006081,0.004962,0.007301,0.005994,0.005046,0.004038,0.002606
2022-02-10,0.002363,0.004818,NaN,0.001293,0.002117,0.003102,0.005901,0.001679,0.002006,NaN,...,0.002956,0.003334,0.003633,0.005710,0.007289,0.006193,0.002997,0.004796,0.004248,0.001303
2022-02-11,0.001267,0.002409,NaN,0.004687,0.003071,0.003353,0.003728,0.002846,0.001003,NaN,...,0.002533,0.003149,0.003111,0.003740,0.003644,0.005655,0.006948,0.004577,0.003448,0.000652


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-09      0.006806  0.002001  0.000048  0.000003  0.003136  0.000411   
2022-02-10      0.003403  0.001000  0.011789  0.005210  0.001568  0.000206   
2022-02-11      0.001702  0.000500  0.005894  0.002605  0.000784  0.000103   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-09      0.000183  0.000220  0.000106  0.001815  ...  0.001985   
2022-02-10      0.004637  0.000995  0.000053  0.015193  ...  0.000992   
2022-02-11      0.002318  0.000497  0.000027  0.007597  ...  0.000496   

Province_State                                                                \
Admin2            Platte Sheridan  Sublette Sweetwater         Teton   Uinta   
2022-02-09      0.006579   0.0116  0.010001   0.000068  1.089914e-06  0.0072   
2022-02-10      0.003290   0.0058  0.005000   0.000034  5.449568e-07  0.0036   
2022-02-11      0.001645   0.0029  0.002500   0.000017  2.724784e-07  0.0018   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-02-09       0.752941  0.006098  0.000122  
2022-02-10       0.752941  0.003049  0.000061  
2022-02-11       0.752941  0.001524  0.000031  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 